In [1]:
import nltk
import pandas as pd
import numpy as np
import sys

ids = pd.read_csv('car_model_identifier.csv', header=0)
ids = ids.sort_values(by=['model_id'])
data = pd.read_csv('entire_data_Mentioned_4th.csv', header=0).iloc[:, 1:] ## because the first column are indexes

import re
from mlxtend.preprocessing import TransactionEncoder
te = TransactionEncoder()

## Generate a list of model names

In [7]:
unique_model=[]
for x in ids.model_fromsales:
    if x not in unique_model:
         unique_model.append(x)

### Add 416 new columns to the dataframe containing the detected entities and org_id information, each named by a model name.
### Replace the Nan values in the dataframe with 0's
### Convert the model_ids to integers.

In [6]:
data = pd.concat([data, pd.DataFrame(columns=unique_model)])
data = data.replace(np.nan, 0)
data.model_id = data.model_id.astype(int)

### For each detection, if the detected model_id matches with one of the model_id of a unique model name, in the dataframe we add 1 to the cell counting the occurence of that model_id. Then save the dataframe to "Pyshark_model". Where we have all the information of detections, and the number of the occurence of certain model name. 

In [127]:
for i in range(len(data)):
    for y in range (416):
        if data.model_id[i] == y+1:
             data.iloc[i,y+7] = 1 ## because the model columns starts from the 7th column 

855.2531158924103


In [129]:
data.to_csv('PyShark_model2.csv')


### Below we only focus on the part that counts the occurences of model names.

In [9]:
data = pd.read_csv('PyShark_model2.csv', header=0).iloc[:, 1:] ## because the first column are indexes
data_record = data.iloc[0:,6:]

In [12]:
data_record.head()

,org_id,CL,ILX,MDX,NSX,RDX,RL,RSX,TL,TSX,...,30,40,50,60,70 C,70 V,80,XC60,XC70,XC90
0,185.0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,150328.0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1851.0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1858.0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2109.0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Add up the occurences of model names in a single review (group by org_id), and save this condensed file

In [13]:
data_condensed=data_record.groupby(['org_id']).sum()
data_condensed.to_csv("PyShark_condensed_model2.csv")

In [16]:
## Data_condensed is a matrix with all models as its columns, and all comments ids as its rows, and each cell indicates
## the numbers that this model names has been mentioned in this comment. 

data_condensed = pd.read_csv("PyShark_condensed_model2.csv", header=0).iloc[:, 1:] 

### Replace all values > 1 with 1 because later we will perform multiplication on all possible pairs, and the sum of these multiplications indicates the times that they are co-mentioned within one comment. So we need the non-zero cells to be 1, other wise we will other values such as 2 * 3 = 6 (instead of 1* 1 = 1) which does not make sense in our case. 

In [17]:
data_condensed = data_condensed.apply(lambda x: [y if y <= 1 else 1 for y in x])  ## replace all values > 1 with 1 

In [19]:
from itertools import combinations

class lift_calculator:
    def __init__(self, data):
        self.data = data
        self.brand_count = dict(data.sum(axis=0))
        self.pair_brand_count = dict()
        for brand1, brand2 in combinations(data.columns, 2):
            self.pair_brand_count[(brand1, brand2)] = (data[brand1] * data[brand2]).sum()
        self.lift_dict = dict()
        self.total_size = len(data)
        
    def calculate(self):
        for brand1, brand2 in self.pair_brand_count:
            self.lift_dict[(brand1, brand2)] = self.pair_brand_count[(brand1, brand2)] * self.total_size / (self.brand_count[brand1] * self.brand_count[brand2])
    
    def top_n(self, n=10):
        data = pd.DataFrame(self.lift_dict.items()).fillna(0).sort_values(by=1, ascending=False).iloc[:n, :]
        top_n_pairs = dict(zip(data[0], data[1]))
        return top_n_pairs
    
    def return_csv(self):
        df = pd.DataFrame(index=data.columns, columns=data.columns)
        for brand1 in data.columns:
            for brand2 in data.columns:
                df.loc[brand1, brand2] = self.lift_dict.get((brand1, brand2), 0)
        return df

In [20]:
data = data_condensed
calculator = lift_calculator(data) ## pass the matrix to lift_calculator
calculator.calculate()

/Users/shijia_huang/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in long_scalars
  from ipykernel import kernelapp as app


In [21]:
lift_model_df = calculator.return_csv()

## the matrix is an upper triangular matrix without diagonal
lift_model_df.to_csv("PyShark_paired_model_lift_no_diagonal2.csv")

In [23]:
calculator.top_n(10)

{('STRATUS COUPE', 'STRATUS SEDAN'): 2058.2903225806454,
 ('DART', 'BRZ'): 1519.2142857142858,
 ('LX470', 'CLK CLASS'): 886.2083333333334,
 ('MONTE CARLO', 'RODEO'): 862.2567567567568,
 ('G CLASS', '9-2X'): 818.0384615384615,
 ('ZDX', 'RX400'): 797.5875,
 ('350Z', 'MR2 SPYDER'): 797.5875,
 ('Q45', 'HS250'): 787.7407407407408,
 ('SONIC', 'MAZDA2'): 733.4137931034483,
 ('ZDX', 'X6'): 708.9666666666667}

## we have about 16842 pairs of models that have a nonzero lift

In [25]:
 np.count_nonzero(lift_model_df)

16842

### follow the instruction from Pyshark (the AP and FD methed generates the same results except ordered differently) to calculate the lift and cross-validate our results above

### https://pyshark.com/market-basket-analysis-using-association-rule-mining-in-python/

In [27]:
from mlxtend.frequent_patterns import apriori
frequent_itemsets_ap = apriori(data_condensed,min_support=0.001, use_colnames=True) ## the min_support could be modified

In [28]:
frequent_itemsets_ap

,support,itemsets
0,0.002805,(CL)
1,0.013400,(MDX)
2,0.001583,(RDX)
3,0.003746,(RL)
4,0.004294,(RSX)
...,...,...
276,0.001395,"(PRIUS C, FR-S)"
277,0.001912,"(PRIUS C, CAMRY)"
278,0.001019,"(70 V, PASSAT)"
279,0.001379,"(ALTIMA, ACCORD, CAMRY)"


In [29]:
from mlxtend.frequent_patterns import fpgrowth
frequent_itemsets_fp=fpgrowth(data_condensed, min_support=0.001, use_colnames=True) ## the min_support could be modified

In [30]:
frequent_itemsets_fp

,support,itemsets
0,0.013306,(5 SERIES)
1,0.002805,(CL)
2,0.006723,(G35/37)
3,0.002335,(SL CLASS)
4,0.011801,(MUSTANG)
...,...,...
276,0.001442,"(F SERIES, C-MAX)"
277,0.001363,"(SONATA, CAMRY)"
278,0.001097,"(SONATA, ACCORD)"
279,0.001113,"(RAM PICKUP, RAM WAGON)"


### Evaluate the lift based on the mlxtend guide:
### http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/

In [31]:
from mlxtend.frequent_patterns import association_rules

rules_ap = association_rules(frequent_itemsets_ap, metric="lift", min_threshold=1.1)
rules_fp = association_rules(frequent_itemsets_fp, metric="lift", min_threshold=1.1) # the min_threshold could be modified


### The two Pyshark methods generate the same results as we did using the dictionary we defined

In [32]:
rules_ap.head(20)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(X5),(MDX),0.006770,0.013400,0.001113,0.164352,12.265262,0.001022,1.180641
1,(MDX),(X5),0.013400,0.006770,0.001113,0.083041,12.265262,0.001022,1.083178
2,(7 SERIES),(TL),0.017051,0.016158,0.001128,0.066176,4.095560,0.000853,1.053563
3,(TL),(7 SERIES),0.016158,0.017051,0.001128,0.069835,4.095560,0.000853,1.056747
4,(CR-V),(TL),0.054477,0.016158,0.001034,0.018987,1.175097,0.000154,1.002884
5,(TL),(CR-V),0.016158,0.054477,0.001034,0.064016,1.175097,0.000154,1.010191
6,(G35/37),(TL),0.006723,0.016158,0.001254,0.186480,11.540971,0.001145,1.209364
7,(TL),(G35/37),0.016158,0.006723,0.001254,0.077595,11.540971,0.001145,1.076833
8,(7 SERIES),(A4),0.017051,0.007742,0.001160,0.068015,8.785049,0.001028,1.064671
9,(A4),(7 SERIES),0.007742,0.017051,0.001160,0.149798,8.785049,0.001028,1.156135
